# Test Fusion on BigStitcher Example Data

Here, we test the calm-utils image fusion code using the (already aligned) 3D dataset from BigStitcher: https://imagej.net/plugins/bigstitcher/#example-datasets

In [ ]:
from xml.etree import ElementTree as et
from pathlib import Path

from h5py import File
import numpy as np

## Parse transforms from XML dataset definition

In [ ]:
# path of XML dataset definition
dataset = Path('/Users/david/Desktop/scratch_data/grid-3d-stitched-h5/dataset.xml')

# wheter to preserve anisotropy (un-do scaling to isotropic coordinates)
preserve_anisotropy = True

# parse XML
dataset_xml = et.parse(dataset)

# get path of H5 data file (NOTE: we assume it is relative to XML)
h5file = dataset.parent / dataset_xml.find('SequenceDescription/ImageLoader/hdf5').text

transforms = {}

for vr in dataset_xml.iterfind('./ViewRegistrations/'):

    # (tp, setup) view id tuple
    view_id = tuple(map(int, vr.attrib.values()))

    # load and accumulate transforms
    tr = np.eye(4)
    for vt in vr.iterfind('ViewTransform/affine'):
        tr_i = np.eye(4)
        tr_i[:3] = np.fromstring(vt.text, sep=' ').reshape((3,4))
        tr = tr @ tr_i
    
    # undo calibration scale to relative pixel sizes
    # (we pre-concatenate inverse transform of the last transform in list := calibration)
    if preserve_anisotropy:
        tr = np.linalg.inv(tr_i) @ tr

    # shuffle to zyx
    tr = tr[:, [2,1,0,3]][[2,1,0,3]]

    transforms[view_id] = tr

## Load images from H5

Here, we load image data (at full resolution) for all view ids we found in the XML dataset before:

In [ ]:
from src.calmutils.misc.string_utils import pad

images = {}

with File(h5file) as reader:
    for (tp, setup) in transforms.keys():
        img = reader[f'/t{pad(tp, 5)}/s{pad(setup, 2)}/0/cells'][...]
        images[(tp, setup)] = img


### Visualize one image in napari

In [ ]:
import napari

if napari.current_viewer() is not None:
    napari.current_viewer().close()

napari.view_image(images[(0,0)])

## Do Fusion

In [ ]:
from src.calmutils.stitching.fusion import fuse_image_wrapper
from src.calmutils.stitching.phase_correlation import get_axes_aligned_bbox

# which view ids to fuse
# 0-6: 6 tiles of first channel
view_ids_to_fuse = [(0,s) for s in range(6)]

# estimate bounding box based on transforms
bbox = get_axes_aligned_bbox([images[k].shape for k in view_ids_to_fuse], [transforms[k] for k in view_ids_to_fuse])

# to int and to list of (min, max) tuples
bbox = (b.astype(int) for b in bbox)
list(zip(*bbox))

# fuse, set block size to something other than None to do multi-threaded
fused = fuse_image_wrapper(bbox, [images[k] for k in view_ids_to_fuse], [transforms[k] for k in view_ids_to_fuse], block_size=(128,256,256), interpolation_mode='linear')

### Visualize fusion results

In [ ]:
if napari.current_viewer() is not None:
    napari.current_viewer().close()

napari.view_image(fused)